# N-grams in Bilinear Transformers

Bilinear transformers are great because they are even more linear in nature than the original architecture. This allows us to perform standardized analysis on each component separately (or even together). This notebook in particular focusses on extracting 2-grams from the weights. This notebook is meant as an introduction to the capabilities of bilinear layers and shouldn't be used to draw rigorous conclusions.

In [19]:
%load_ext autoreload
%autoreload 2

from shared.transformer import Transformer, Config
import plotly.express as px
import torch
import pandas as pd
from einops import *

torch.set_grad_enabled(False)

color = dict(color_continuous_midpoint=0, color_continuous_scale="RdBu")
name = "tdooms/TinyStories-2-256" 

config = Config.from_pretrained(name)
model = Transformer.from_pretrained(name, config=config).cuda()

# model.center_unembed().fold_norms()
vocab = model.vocab

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1 and 2 layer transformers have slightly different behavior. The 1-layer transformer has a slightly more diverse MLP layer (because it kinda has to). Results shown in this notebook hold for both.

## Direct Path

Let's start with the obvious way to look at 2-grams, the direct embedding-unembedding path. 

In [15]:
direct = (model.w_u @ model.w_e).detach().cpu()
assert direct.shape == (len(vocab), len(vocab))

vocab.get_max_activations(direct.T, ["input", "output"], 10)

,input,output,value
0,fl,##inary,3.353223
1,sc,##ience,3.137903
2,believe,forth,3.120463
3,ru,##ss,3.061916
4,ru,##ce,2.968355
5,rec,##ving,2.854641
6,ru,##ence,2.846766
7,ent,##hip,2.839122
8,ru,##hip,2.821685
9,disag,##hip,2.810700


I'm a bit surprised that this doesn't make a lot of sense. I would expect some structure.

TODO: why doesn't this work?

## MLP path

Now, onto the good stuff, the MLP. In a normal neural network, we can't study the MLP with SVD or any linear technique. However, bilinear layers actually allow us to do so. In this section, we will limit ourselves to the direct MLP path, aka embedding -> MLP -> unembedding. To our knowledge, this hasn't been done before. To study the direct path, we can take the diagonal over the last two dimensions of the B tensor. I won't go into the math here for brevity, trust me bro. 

Before looking at the eigenvalues, let's look at the highest activations in general, this will result in a map of input -> output, meaning that we get the pairs of which the model is most sure.

In [20]:
diag = model.ube.diagonal(residual=True).cpu()

I use a helper function ``get_max_activations`` This returns a data frame of indices and values of the max values in the provided tensor. The indices are automatically converted to tokens. Let's look at the top 1000 connections in the first MLP layer.

In [21]:
df0 = vocab.get_max_activations(diag[0].T, ["input", "output"], k=1_000, largest=True)
df0

,input,output,value
0,[BOS],once,36.167042
1,[BOS],one,34.639488
2,[BOS],the,30.953783
3,[BOS],it,28.075520
4,[BOS],lily,27.887444
...,...,...,...
995,str,##o,15.276323
996,je,##s,15.271566
997,pete,##eath,15.269166
998,mitt,##let,15.266798


Okay, so it's obvious that the most first layer just connects the obvious bi-grams of words that didn't quite get included in the tokenizer. Let's quantify this.

In [22]:
px.line(df0["output"].str.startswith("##").cumsum(), title="cumulative ## tokens")

So, the first layer seems to only be bothered with learning these kinds of connections. Let's look at the second layer.

In [12]:
df1 = vocab.get_max_activations(diag[1].T, ["input", "output"], k=1_000, largest=True)
df1.head(20)

,input,output,value
0,later,##ct,3.554982
1,happened,##ughter,3.416218
2,one,##day,3.319006
3,either,##day,3.220840
4,feeling,##ort,3.172991
5,one,matter,3.114573
6,excited,##ish,3.112399
7,con,##age,3.069427
8,picture,##ies,3.026420
9,zoo,itself,2.999322


That makes way less sense. Let's try to look if there's a pattern on the tokens it activates strongly on.

In [13]:
input_mean = diag[1].mean(1).view(64, 64)
output_mean = diag[1].mean(0).view(64, 64)

fig = px.imshow(torch.stack((input_mean, output_mean)), facet_col=0, **color)
fig.layout.annotations[0].update(text="input")
fig.layout.annotations[1].update(text="output")

fig

Except from the fact that the inputs seem to mostly looking positively at a handful of tokens, nothing discernable, we'll leave this inspection for later.

#### Preceding and Following tokens

Given this diagonal matrix, we can also analyze which words are most important indicators for the next word or the other way around.
For instance, we can ask:
- *"what tokens are most important for the model to decide to predict the token 'game'"* (preceding token).
- *"what tokens does the token 'game' infer most"* (following token).

In [14]:
token = "girl"
idx = vocab[token]

preceding = vocab.tokenize(torch.topk(diag[0, idx], k=10).indices)
following = vocab.tokenize(torch.topk(diag[0, :, idx], k=10).indices)

pd.DataFrame(dict(preceding=preceding, self=[token]*10, following=following))

,preceding,self,following
0,older,girl,##sy
1,##ular,girl,##c
2,wealthy,girl,##lly
3,independent,girl,##s
4,new,girl,##p
5,naughty,girl,##ss
6,helpful,girl,##ath
7,busy,girl,##me
8,young,girl,##hood
9,careless,girl,##ces


Left and right are not related, this is simply a concise visualization. 

#### Articles

Something interesting to look at is if the model has learned to use correct articles. Let's study this a bit more in-depth.
We can do this quite simply by taking the weights for both for all subsequent tokens and plotting them together.

In [15]:
mask_non_words = torch.tensor([vocab.inv[idx][0].isalpha() for idx in range(len(vocab))])

vowels = ['a', 'e', 'i', 'o', 'u']

token = vocab.tokenize(torch.arange(len(vocab)))
df = pd.DataFrame(dict(x=diag[0, :, vocab["a"]].cpu(), y=diag[0, :, vocab["an"]].cpu(), token=token))
df = df[df.token.str[0].str.isalpha()]
df["guess"] = df.token.str[0].isin(vowels)

px.scatter(df, x="x", y="y", hover_name="token", color="guess", labels=dict(x="a", y="an")).show()

The result isn't as clean as I'd hoped but it seems that the model simply generally has a strong bias towards picking 'a' which is sensible. If you hover over most tokens, it's clear why it's "unsure" about some of them, a proper filtering of verbs and such will probably improve the separation. Alos, I'd assume this becomes more clear as models improve.

## Token Interactions
Until now, we've only looked at the direct path. This is fine, but the MLP encodes so much more information than (input output)-pairs. Specifically, it actually encodes (input, input, output)-triplets, being one of the reasons for its effectiveness.

So, in essence, until now, we've just looked at token interactions with itself. This reduces the UBE tensor to a matrix, which we can study. Now, we will perform another reduction, by just taking the first dimension of the UBE tensor, which means that we will get the input-input interactions for a certain token.

In [16]:
idx = vocab["boy"]
inter = model.ube.interaction(idx, residual=True).cpu()[0]

topk = torch.topk(inter.tril().flatten(), k=25)
input1, input2 = torch.unravel_index(topk.indices, inter.size())
pd.DataFrame(dict(input1=vocab.tokenize(input1), input2=vocab.tokenize(input2), value=topk.values.cpu()))

,input1,input2,value
0,the,the,1.136929
1,boy,boy,1.035445
2,little,little,0.939118
3,son,boy,0.928262
4,son,son,0.923083
5,boy,man,0.860749
6,gra,pl,0.844902
7,man,man,0.842944
8,man,he,0.816335
9,its,the,0.810491


This will mostly become useful once we introduce some additional inspection techniques in later attention layers.